In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import json
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input, InputLayer
from tensorflow.keras import optimizers
import tensorflow.keras.backend as K
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import nasnet, resnet50, densenet
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

%matplotlib inline

In [29]:
df=pd.read_csv('Kaggle-assignment-2/train.csv')
df.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


In [30]:
df.dtypes

id                         int64
belongs_to_collection     object
budget                     int64
genres                    object
homepage                  object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity               float64
poster_path               object
production_companies      object
production_countries      object
release_date              object
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
Keywords                  object
cast                      object
crew                      object
revenue                    int64
dtype: object

In [31]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,3000.0,1.500500e+03,8.661697e+02,1.000000,7.507500e+02,1.500500e+03,2.250250e+03,3.000000e+03
budget,3000.0,2.253133e+07,3.702609e+07,0.000000,0.000000e+00,8.000000e+06,2.900000e+07,3.800000e+08
popularity,3000.0,8.463274e+00,1.210400e+01,0.000001,4.018053e+00,7.374861e+00,1.089098e+01,2.943370e+02
runtime,2998.0,1.078566e+02,2.208643e+01,0.000000,9.400000e+01,1.040000e+02,1.180000e+02,3.380000e+02
revenue,3000.0,6.672585e+07,1.375323e+08,1.000000,2.379808e+06,1.680707e+07,6.891920e+07,1.519558e+09


In [32]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,3000,NaN,NaN,NaN,1500.5,866.17,1,750.75,1500.5,2250.25,3000
belongs_to_collection,604,422,"[{'id': 645, 'name': 'James Bond Collection', ...",16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
budget,3000,NaN,NaN,NaN,2.25313e+07,3.70261e+07,0,0,8e+06,2.9e+07,3.8e+08
genres,2993,872,"[{'id': 18, 'name': 'Drama'}]",266,NaN,NaN,NaN,NaN,NaN,NaN,NaN
homepage,946,941,http://www.transformersmovie.com/,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
imdb_id,3000,3000,tt0105508,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
original_language,3000,36,en,2575,NaN,NaN,NaN,NaN,NaN,NaN,NaN
original_title,3000,2975,The Women,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
overview,2992,2992,John Hall is a drifter who wanders into a smal...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
popularity,3000,NaN,NaN,NaN,8.46327,12.104,1e-06,4.01805,7.37486,10.891,294.337


In [33]:
df['original_language'].value_counts()

en    2575
fr      78
ru      47
es      43
hi      42
ja      37
it      24
ko      20
cn      20
zh      19
de      18
ta      16
sv       8
pt       6
nl       6
fa       5
da       5
ro       4
tr       3
hu       3
fi       2
pl       2
no       2
te       2
ml       2
he       1
sr       1
vi       1
el       1
ur       1
nb       1
ar       1
id       1
bn       1
mr       1
cs       1
Name: original_language, dtype: int64

In [34]:
df.corr()

,id,budget,popularity,runtime,revenue
id,1.000000,0.019732,-0.007470,0.010750,0.000610
budget,0.019732,1.000000,0.342356,0.238373,0.752965
popularity,-0.007470,0.342356,1.000000,0.133690,0.461460
runtime,0.010750,0.238373,0.133690,1.000000,0.216380
revenue,0.000610,0.752965,0.461460,0.216380,1.000000


In [46]:
df['release_date'] = pd.to_datetime(df['release_date'])
df['release_year'] = df['release_date'].dt.year
df['release_2000'] = np.where(df['release_year'] >= 2000, 1, 0)
df = df[df.release_year <= 2019]

In [47]:
df.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,tagline,title,Keywords,cast,crew,revenue,date,year,release_year,release_2000
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651,2015-02-20,2015,2015,1
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435,2004-08-06,2004,2004,1
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000,2014-10-10,2014,2014,1
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000,2012-03-09,2012,2012,1
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970,2009-02-05,2009,2009,1


In [49]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,2625.0,1.496679e+03,8.693992e+02,1.000000,7.480000e+02,1.498000e+03,2.251000e+03,3.000000e+03
budget,2625.0,2.500951e+07,3.883054e+07,0.000000,0.000000e+00,1.000000e+07,3.200000e+07,3.800000e+08
popularity,2625.0,8.689973e+00,1.279701e+01,0.000001,4.235440e+00,7.452790e+00,1.097819e+01,2.943370e+02
runtime,2623.0,1.071697e+02,2.119419e+01,0.000000,9.400000e+01,1.040000e+02,1.170000e+02,3.380000e+02
revenue,2625.0,7.186646e+07,1.449084e+08,1.000000,2.225098e+06,1.765491e+07,7.657864e+07,1.519558e+09
year,2625.0,2.004161e+03,8.900195e+00,1985.000000,1.998000e+03,2.006000e+03,2.012000e+03,2.017000e+03
release_year,2625.0,2.004161e+03,8.900195e+00,1985.000000,1.998000e+03,2.006000e+03,2.012000e+03,2.017000e+03
release_2000,2625.0,7.009524e-01,4.579280e-01,0.000000,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
